In [1]:
import Get_data as Gd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

directory = '..\\data\\Sibo_22Mar2024'
folders = [os.path.join(directory, f, 'hessian_') for f in os.listdir(directory) if f.startswith('case_')]
train_set, length = Gd.get_all_data(folders[:-2])
# test_set, length_test = Gd.get_all_data(folders[-2:])

In [2]:
preprocessor = Gd.PreprocessorCNN(train_set, 3)
preprocessor.fit()

tt = preprocessor.transform(train_set)


In [3]:
mm = preprocessor.inverse_transform(tt)
mm.shape

MemoryError: Unable to allocate 2.03 GiB for an array with shape (2800, 97149) and data type float64

In [3]:
tt.max()

10.304402208942674

In [94]:
class Conv1DAutoencoder(nn.Module):
    def __init__(self):
        super(Conv1DAutoencoder, self).__init__()
        
        # 编码器部分
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # Output size: (97800 / 2) = 48900
            
            nn.Conv1d(in_channels=16, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # Output size: (97800 / 2) = 48900
            
            nn.Conv1d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # Output size: (97800 / 2) = 48900
        )
        self.flatten = nn.Flatten()
        self.fc1 = None
        self.fc2 = nn.Linear(1000, 100)
        self.fc3 = nn.Linear(100, 1000)
        self.fc4 = None
        
        # 解码器部分
        # 解码器部分
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=256, out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1),  # 解压
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=64, out_channels=16, kernel_size=3, stride=2, padding=1, output_padding=1),  # 继续解压
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=16, out_channels=3, kernel_size=3, stride=2, padding=1, output_padding=1),  # 继续解压
            nn.ReLU()
        )

    def forward(self, x):
        x = self.encoder(x)
        encoded_shape = x.shape
        x = self.flatten(x)
        num = x.shape[1]
        if self.fc1 is None:
            self.fc1 = nn.Linear(num, 1000)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        if self.fc4 is None:
            self.fc4 = nn.Linear(1000, num)
        x = self.fc4(x)
        x = x.view(encoded_shape)
        x = self.decoder(x)
        return x

In [97]:
def train(model, optimizer, criterion, folders, epochs=100):
    for epoch in range(epochs):
        model.train()
        test_datasets = []
        total_loss = 0
        for folder in folders:
            files = [os.path.join(folder, f) for f in os.listdir(folder) if f.startswith('foward_soln_timestep_') and f.endswith('.npy')]
            data = get_data(files)
            train_set, test_set = create_dataset(data, 0.2)
            test_datasets.append(test_set)
            train_loader = DataLoader(train_set, batch_size=10, shuffle=True)
            loss_of_folder = 0
            for input in train_loader:
                optimizer.zero_grad()
                output = model(input)
                loss = criterion(input, output)  # 计算损失
                loss.backward()
                optimizer.step()  # 更新参数
                loss_of_folder += loss.item()  # 累计损失
            loss_of_folder = loss_of_folder / len(train_loader)
            total_loss += loss_of_folder
        # 在每个epoch后打印平均损失
        print(f'Epoch {epoch + 1}/{epochs}, train Loss: {total_loss / len(folders)}')
        
        #evaluate the model
        recon_err = 0
        model.eval()
        with torch.no_grad():
            for test_data in test_datasets:
                scaler = TensorNormalizer()
                scaler.fit(test_data)
                transformed_data = scaler.transform(test_data)
                err = 0
                for i in range(transformed_data.shape[0]):
                    output = model(transformed_data[i].unsqueeze(0))
                    reconstruction = scaler.inverse_transform(output.squeeze())
                    err += criterion(test_data[i], reconstruction).item()
                recon_err += err/transformed_data.shape[0]
            
            recon_err = recon_err / len(test_datasets)
            print(f'val Loss: {recon_err}')

In [98]:
import torch.optim as optim
criterion = nn.MSELoss()
model = Conv1DAutoencoder()
optimizer = optim.Adam(model.parameters())

train(model, optimizer, criterion, folders)

KeyboardInterrupt: 